In [1]:
import os, sys
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import random

import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer

from indicators import *


physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
top100tech = ['AAPL', 'MSFT', 'TSM', 'NVDA', 'INTC', 'ASML', 'ADBE', 'CRM', 'AVGO', 'ORCL', 'CSCO', 'ACN', 'TXN', 'SHOP', 'QCOM', 'SAP', 'SNE', 'AMAT', 'INTU', 'NOW', 'SQ', 'IBM', 'MU', 'AMD',
'UBER', 'FIS', 'LRCX', 'FISV', 'SNOW', 'INFY', 'TEAM', 'ADSK', 'DELL', 'WDAY', 'VMW', 'ADI', 'NXPI', 'KLAC',
'CRWD', 'PLTR', 'DOCU', 'TEL', 'ERIC', 'MCHP', 'CDNS', 'CTSH',
'APH', 'SNPS', 'HPQ',
'PANW', 'OKTA','RNG','STM','MRVL','XLNX','WIT','MSI','SWKS','GLW','DDOG','ANSS','U','ZS','FTNT','STNE','ZBRA','KEYS','FTV','MXIM','COUP','GRMN','SPLK','HUBS','WORK','NET','FLT','CDW','CAJ','AFRM','PAYC','TER',
'VRSN','WDC', 'ZI',
'UMC','ANET','EPAM',
'UI','XM',
'GIB','TRMB',
'TYL','WIX','HPE','LYFT',
'LOGI',
'MPWR']

In [3]:
top10tech = top100tech[:10]

In [4]:
data = yf.download(tickers = ' '.join(top10tech),
                   auto_adjust=True,
                  period='4y')

[*********************100%***********************]  10 of 10 completed


In [5]:
data.head(8)

Close                                                                                                              High                                                                                                               Low                                                                                                              Open                                                                                                            Volume                                                                                     
                 AAPL        ADBE        ASML        AVGO        CRM       INTC       MSFT        NVDA       ORCL        TSM       AAPL        ADBE        ASML        AVGO        CRM       INTC       MSFT        NVDA       ORCL        TSM       AAPL        ADBE        ASML        AVGO        CRM       INTC       MSFT        NVDA       ORCL        TSM       AAPL        ADBE        ASML        AVGO        CRM       INTC       MSFT        NVDA       ORCL        TSM       AAPL     ADBE     ASML     AVGO      CRM      INTC      MSFT      NVDA      ORCL      TSM
Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
2017-03-13  33.052044  121.800003  118.892601  195.369949  83.610001  31.785255  60.897831  100.739799  39.855618  27.374693  33.106655  121.959999  119.151081  196.491528  83.739998  32.580792  61.349557  101.135441  40.042818  27.435525  32.961818  120.889999  118.165017  194.429554  83.080002  31.586370  60.766079   98.425310  39.808814  27.148743  32.968941  121.110001  118.289469  194.860929  83.470001  32.409026  61.180160   98.454982  40.014739  27.174815   69686800  1805200   838100  2666900  3463700  53582900  20100000  19353400  12440800  3965700
2017-03-14  33.002182  121.440002  118.701118  193.558197  83.510002  31.803339  60.615509  100.670547  40.052181  27.122673  33.158892  121.989998  119.505291  195.223300  83.620003  31.848539  60.747261  101.629975  40.098981  27.348623  32.966563  120.620003  118.576666  192.833476  82.730003  31.333249  60.370824   99.374830  39.696494  27.113984  33.075789  121.650002  119.447846  193.911915  83.419998  31.694858  60.728435  101.254120  39.855618  27.270410   61236400  1507500   412900  2216500  2933500  28586900  14280200  13050400  10291000  5967200
2017-03-15  33.351227  122.110001  120.692398  194.921356  83.480003  31.731012  60.935478  101.432175  40.295540  27.730999  33.420084  122.419998  120.922159  195.542538  83.769997  31.794293  61.095462  101.867373  40.435944  27.835283  33.011682  120.900002  119.265957  192.600567  83.000000  31.351327  60.464934   99.226479  39.911773  27.235648  33.101911  122.000000  119.524437  193.540962  83.559998  31.586370  60.747263  101.135442  40.155139  27.270408  102767200  2396300   838900  2072700  3074100  27996100  24833800  15375900  18393200  4571400
2017-03-16  33.405842  122.349998  120.807289  192.711334  83.519997  31.767178  60.831951  102.678436  42.804070  28.165514  33.484198  123.599998  120.864727  195.709996  84.000000  31.966063  60.944884  102.698222  43.983453  28.217657  33.303739  121.529999  119.486149  192.026676  83.269997  31.703897  60.511985  101.293703  42.729190  27.983017  33.412965  123.250000  119.562739  195.415321  83.949997  31.794297  60.935471  101.857489  43.421841  28.026468   76928000  4044600   424200  2783500  3762100  19616100  20674300  13077500  67535400  6489000
2017-03-17  33.239635  127.010002  122.444344  191.688660  83.029999  31.88

In [6]:
data.shape

(1007, 50)

In [7]:
df = pd.melt(data, ignore_index=False)
df.columns = ['Variable', 'Ticker', 'Value']

In [8]:
# this is a nicely formed dataframe of historical data
nice_df = df.reset_index().pivot(index=['Ticker', 'Date'], columns=['Variable'], values=['Value'])

In [9]:
nice_df.columns = nice_df.columns.droplevel()

In [10]:
nice_df.head(5)

Variable               Close       High        Low       Open       Volume
Ticker Date                                                               
AAPL   2017-03-13  33.052044  33.106655  32.961818  32.968941   69686800.0
       2017-03-14  33.002182  33.158892  32.966563  33.075789   61236400.0
       2017-03-15  33.351227  33.420084  33.011682  33.101911  102767200.0
       2017-03-16  33.405842  33.484198  33.303739  33.412965   76928000.0
       2017-03-17  33.239635  33.479452  33.215890  33.479452  175540000.0

In [11]:
df = nice_df.reset_index()
df.columns.name=None

In [12]:
df.head()

,Ticker,Date,Close,High,Low,Open,Volume
0,AAPL,2017-03-13,33.052044,33.106655,32.961818,32.968941,69686800.0
1,AAPL,2017-03-14,33.002182,33.158892,32.966563,33.075789,61236400.0
2,AAPL,2017-03-15,33.351227,33.420084,33.011682,33.101911,102767200.0
3,AAPL,2017-03-16,33.405842,33.484198,33.303739,33.412965,76928000.0
4,AAPL,2017-03-17,33.239635,33.479452,33.215890,33.479452,175540000.0


In [13]:
df.shape

(10070, 7)

In [14]:
# compute all our technical markers for each of the stocks. 
dataset = df.groupby('Ticker').apply(lambda x: add_technical_markers(x, 14))

In [15]:
dataset.shape

(10070, 31)

In [16]:
dataset.head(16)

,Ticker,Date,Close,High,Low,Open,Volume,HH,LL,AV,SMA,SD,WILLR,ATR,DMH,DML,EMA,WMA,BBHIGH,BBLOW,PERBHIGH,PERBLOW,TRIMA,RSI,DX,PDI,NDI,ADX,ROC,MACD,CCI
0,AAPL,2017-03-13,33.052044,33.106655,32.961818,32.968941,69686800.0,33.106655,32.961818,6.968680e+07,33.052044,NaN,-37.705123,0.144837,0.000000,0.000000,33.052044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN
1,AAPL,2017-03-14,33.002182,33.158892,32.966563,33.075789,61236400.0,33.158892,32.961818,6.546160e+07,33.027113,0.035258,-81.480295,0.168582,0.052237,0.004745,33.025332,NaN,33.097628,32.956598,938.734934,934.734934,NaN,NaN,83.344552,30.985901,2.814832,83.344552,-0.150859,-0.000890,44.894230
2,AAPL,2017-03-15,33.351227,33.420084,33.011682,33.101911,102767200.0,33.420084,32.961818,7.789680e+07,33.135151,0.188781,-16.860063,0.251689,0.261192,0.045118,33.149825,NaN,33.512712,32.757590,177.521933,173.521933,NaN,NaN,70.540847,103.775817,17.926191,76.942699,0.905187,0.007441,19.006587
3,AAPL,2017-03-16,33.405842,33.484198,33.303739,33.412965,76928000.0,33.484198,32.961818,7.765460e+07,33.202824,0.205127,-43.420736,0.233881,0.064115,0.292058,33.228148,NaN,33.613078,32.792569,163.864577,159.864577,NaN,NaN,63.997924,27.413378,124.874384,72.627774,1.070427,0.012680,41.415027
4,AAPL,2017-03-17,33.239635,33.479452,33.215890,33.479452,175540000.0,33.484198,32.961818,9.723168e+07,33.210186,0.178407,-90.990446,0.239817,-0.004747,-0.087850,33.231145,NaN,33.566999,32.853373,188.148935,184.148935,NaN,NaN,89.747132,-1.979384,-36.631938,76.907614,0.567564,0.010130,34.250020
5,AAPL,2017-03-20,33.588676,33.598173,33.296619,33.336984,86168000.0,33.598173,32.961818,9.538773e+07,33.273268,0.222124,-3.149067,0.259604,0.118721,0.080729,33.313871,NaN,33.717515,32.829020,151.796100,147.796100,NaN,NaN,19.048211,45.731594,31.097101,65.335733,1.623599,0.020136,34.841057
6,AAPL,2017-03-21,33.204002,33.906835,33.177884,33.742999,158119600.0,33.906835,32.961818,1.043494e+08,33.263373,0.204453,-96.416927,0.326654,0.308662,-0.118736,33.290720,NaN,33.672279,32.854466,164.694187,160.694187,NaN,NaN,225.033073,94.492179,-36.349062,91.951956,0.459755,0.012707,60.646578
7,AAPL,2017-03-22,33.579170,33.621912,33.185014,33.206387,103440800.0,33.906835,32.961818,1.042358e+08,33.302847,0.219763,-9.782978,0.340434,-0.284923,0.007131,33.347136,NaN,33.742373,32.863322,153.540122,149.540122,NaN,NaN,105.133814,-83.693982,2.094581,93.835079,1.594837,0.021424,20.348349
8,AAPL,2017-03-23,33.460449,33.617162,33.386843,33.541179,81385200.0,33.906835,32.961818,1.016969e+08,33.320359,0.212176,-68.041529,0.328199,-0.004749,0.201828,33.368000,NaN,33.744710,32.896008,159.041477,155.041477,NaN,NaN,104.819840,-1.447122,61.495673,95.208174,1.235643,0.022698,38.048946
9,AAPL,2017-03-24,33.393963,33.655151,33.325106,33.598164,89582400.0,33.906835,32.961818,1.004854e+08,33.327719,0.201391,-79.137164,0.328384,0.037989,-0.061736,33.372549,NaN,33.730500,32.924938,167.487981,163.487981,NaN,NaN,419.942710,11.568494,-18.800098,131.289789,1.034487,0.020675,35.841824


In [17]:
dataset.describe()

,Close,High,Low,Open,Volume,HH,LL,AV,SMA,SD,WILLR,ATR,DMH,DML,EMA,WMA,BBHIGH,BBLOW,PERBHIGH,PERBLOW,TRIMA,RSI,DX,PDI,NDI,ADX,ROC,MACD,CCI
count,10070.000000,10070.000000,10070.000000,10070.000000,1.007000e+04,10070.000000,10070.000000,1.007000e+04,10070.000000,10060.000000,1.007000e+04,10070.000000,10070.000000,10070.000000,10070.000000,9940.000000,10060.000000,10060.000000,10060.000000,10060.000000,9940.000000,9930.000000,1.005900e+04,10070.000000,10070.000000,1.006000e+04,10070.000000,10070.000000,10060.000000
mean,154.081985,155.925061,152.096101,154.096096,2.337662e+07,161.103898,143.741798,2.333263e+07,152.826239,4.320528,-4.695725e+01,4.261525,0.198451,0.193281,152.802100,153.325135,161.538357,144.256245,56.554170,52.554170,152.393767,55.639058,1.010384e+04,6.541375,5.851241,1.025063e+04,1.882205,1.421236,14.568333
std,118.810495,120.435114,117.062448,118.905458,4.130951e+07,124.782753,110.124939,3.866042e+07,117.548875,4.860329,3.074039e+01,4.476919,3.847540,4.074453,117.412887,117.525338,125.020950,110.482122,54.340814,54.340814,116.478129,11.577926,3.678467e+05,66.931667,70.053303,9.837195e+04,7.065935,4.050966,51.879021
min,27.122673,27.348623,27.113984,27.174815,1.913000e+05,27.435525,27.113984,4.171571e+05,27.248683,0.012787,-1.000000e+02,0.144837,-43.387152,-46.946157,27.239682,27.982607,27.605093,26.692518,8.037656,4.037656,27.866110,18.890778,0.000000e+00,-449.429295,-544.398188,7.499858e+00,-38.722400,-25.861245,-706.977168
25%,50.888574,51.414989,50.386837,50.918787,3.070800e+06,53.028847,48.144104,3.332370e+06,50.801071,1.083277,-7.409453e+01,1.069155,-0.680006,-0.662277,50.747849,50.865024,53.172751,48.262801,33.084252,29.084252,50.679714,47.908211,1.516447e+01,-30.823425,-31.381830,5.552566e+01,-1.738001,-0.088732,-25.619831
50%,130.834999,132.355003,129.281779,130.709010,8.782950e+06,136.837152,122.789850,9.230675e+06,130.095806,2.601316,-4.561997e+01,2.807597,0.096192,0.163886,129.724490,131.507546,136.587181,122.851656,47.458740,43.458740,130.805332,55.892489,3.603631e+01,5.840336,10.309134,8.288090e+01,2.149861,0.699336,24.825192
75%,218.603249,221.960142,215.651160,219.220513,2.191515e+07,230.300003,204.343185,2.307110e+07,218.038780,5.878465,-1.857456e+01,5.786545,1.049323,1.185702,217.950951,218.864893,231.161501,205.071229,69.218650,65.218650,217.964077,63.853186,8.066485e+01,42.717734,47.057187,1.434262e+02,5.847236,2.591635,55.457915
max,613.210022,614.900024,599.169983,606.840027,4.268848e+08,614.900024,559.659973,3.155055e+08,582.634290,41.908829,1.312337e-12,34.143489,59.619995,38.669998,579.551148,589.851623,635.333070,554.851308,2488.451536,2484.451536,574.697402,92.784135,2.327154e+07,607.968514,432.043602,1.662365e+06,53.437230,30.346531,162.709976


In [18]:
X = dataset['Close']

In [19]:
tickers = dataset['Ticker'].unique()

In [20]:
dataset.groupby("Ticker").apply(lambda x: x)

,Ticker,Date,Close,High,Low,Open,Volume,HH,LL,AV,SMA,SD,WILLR,ATR,DMH,DML,EMA,WMA,BBHIGH,BBLOW,PERBHIGH,PERBLOW,TRIMA,RSI,DX,PDI,NDI,ADX,ROC,MACD,CCI
0,AAPL,2017-03-13,33.052044,33.106655,32.961818,32.968941,69686800.0,33.106655,32.961818,6.968680e+07,33.052044,NaN,-37.705123,0.144837,0.000000,0.000000,33.052044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN
1,AAPL,2017-03-14,33.002182,33.158892,32.966563,33.075789,61236400.0,33.158892,32.961818,6.546160e+07,33.027113,0.035258,-81.480295,0.168582,0.052237,0.004745,33.025332,NaN,33.097628,32.956598,938.734934,934.734934,NaN,NaN,83.344552,30.985901,2.814832,83.344552,-0.150859,-0.000890,44.894230
2,AAPL,2017-03-15,33.351227,33.420084,33.011682,33.101911,102767200.0,33.420084,32.961818,7.789680e+07,33.135151,0.188781,-16.860063,0.251689,0.261192,0.045118,33.149825,NaN,33.512712,32.757590,177.521933,173.521933,NaN,NaN,70.540847,103.775817,17.926191,76.942699,0.905187,0.007441,19.006587
3,AAPL,2017-03-16,33.405842,33.484198,33.303739,33.412965,76928000.0,33.484198,32.961818,7.765460e+07,33.202824,0.205127,-43.420736,0.233881,0.064115,0.292058,33.228148,NaN,33.613078,32.792569,163.864577,159.864577,NaN,NaN,63.997924,27.413378,124.874384,72.627774,1.070427,0.012680,41.415027
4,AAPL,2017-03-17,33.239635,33.479452,33.215890,33.479452,175540000.0,33.484198,32.961818,9.723168e+07,33.210186,0.178407,-90.990446,0.239817,-0.004747,-0.087850,33.231145,NaN,33.566999,32.853373,188.148935,184.148935,NaN,NaN,89.747132,-1.979384,-36.631938,76.907614,0.567564,0.010130,34.250020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10065,TSM,2021-03-05,120.800003,121.300003,115.379997,120.900002,17420100.0,142.199997,113.489998,1.352314e+07,129.209288,6.840381,-8.445938,4.993574,-1.849998,1.889999,126.526964,132.523527,142.890051,115.528525,20.889193,16.889193,132.352553,42.825375,9349.780660,-37.047580,37.848628,691.157208,-12.501807,-0.324681,-70.621716
10066,TSM,2021-03-08,113.930000,121.000000,113.699997,120.540001,17199900.0,139.639999,113.489998,1.390645e+07,127.343574,7.208348,-96.849270,5.219289,-0.300003,-1.680000,124.847369,130.829337,141.760271,112.926877,19.666124,15.666124,132.143369,36.916915,69.696713,-5.747968,-32.188298,695.935844,-18.650484,-1.113128,-61.617292
10067,TSM,2021-03-09,116.379997,118.400002,115.330002,117.070000,17640800.0,137.949997,113.489998,1.428714e+07,125.922859,7.273474,-65.798191,5.256432,-2.599998,1.630005,123.718386,129.449718,140.469808,111.375911,19.312615,15.312615,131.758573,40.091277,436.085701,-49.463180,31.009720,726.161601,-14.596027,-1.581387,-55.347889
10068,TSM,2021-03-10,113.830002,118.029999,113.410004,117.800003,15185300.0,137.949997,113.410004,1.454219e+07,124.437145,7.479534,-90.909121,5.381431,-0.370003,-1.919998,122.399935,128.090670,139.396212,109.478077,18.637019,14.637019,131.222757,37.950742,67.685363,-6.875546,-35.678208,729.824243,-15.449753,-2.108820,-51.798497


In [21]:
stock = dataset.loc[dataset['Ticker']=='AAPL']

In [22]:
stock

,Ticker,Date,Close,High,Low,Open,Volume,HH,LL,AV,SMA,SD,WILLR,ATR,DMH,DML,EMA,WMA,BBHIGH,BBLOW,PERBHIGH,PERBLOW,TRIMA,RSI,DX,PDI,NDI,ADX,ROC,MACD,CCI
0,AAPL,2017-03-13,33.052044,33.106655,32.961818,32.968941,69686800.0,33.106655,32.961818,6.968680e+07,33.052044,NaN,-37.705123,0.144837,0.000000,0.000000,33.052044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN
1,AAPL,2017-03-14,33.002182,33.158892,32.966563,33.075789,61236400.0,33.158892,32.961818,6.546160e+07,33.027113,0.035258,-81.480295,0.168582,0.052237,0.004745,33.025332,NaN,33.097628,32.956598,938.734934,934.734934,NaN,NaN,83.344552,30.985901,2.814832,83.344552,-0.150859,-0.000890,44.894230
2,AAPL,2017-03-15,33.351227,33.420084,33.011682,33.101911,102767200.0,33.420084,32.961818,7.789680e+07,33.135151,0.188781,-16.860063,0.251689,0.261192,0.045118,33.149825,NaN,33.512712,32.757590,177.521933,173.521933,NaN,NaN,70.540847,103.775817,17.926191,76.942699,0.905187,0.007441,19.006587
3,AAPL,2017-03-16,33.405842,33.484198,33.303739,33.412965,76928000.0,33.484198,32.961818,7.765460e+07,33.202824,0.205127,-43.420736,0.233881,0.064115,0.292058,33.228148,NaN,33.613078,32.792569,163.864577,159.864577,NaN,NaN,63.997924,27.413378,124.874384,72.627774,1.070427,0.012680,41.415027
4,AAPL,2017-03-17,33.239635,33.479452,33.215890,33.479452,175540000.0,33.484198,32.961818,9.723168e+07,33.210186,0.178407,-90.990446,0.239817,-0.004747,-0.087850,33.231145,NaN,33.566999,32.853373,188.148935,184.148935,NaN,NaN,89.747132,-1.979384,-36.631938,76.907614,0.567564,0.010130,34.250020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,AAPL,2021-03-05,121.419998,121.940002,117.570000,120.980003,153590400.0,136.009995,117.570000,1.221485e+08,125.685000,4.164122,-11.899404,4.419999,-1.659996,-1.050003,125.513672,127.504667,134.013243,117.356756,32.182835,28.182835,131.267176,37.633665,22.508974,-37.556478,-23.755729,95.911798,-10.305088,-2.617844,-63.291977
1003,AAPL,2021-03-08,116.360001,121.000000,116.209999,120.930000,153918600.0,132.220001,116.209999,1.273872e+08,124.482857,4.259084,-96.868445,4.562142,-0.940002,-1.360001,124.293182,126.343714,133.001024,115.964690,31.227616,27.227616,130.470491,31.646513,18.260766,-20.604410,-29.810572,73.747418,-12.636085,-3.026505,-61.336693
1004,AAPL,2021-03-09,121.089996,122.059998,118.790001,119.029999,129159600.0,130.710007,116.209999,1.296187e+08,123.786428,3.923552,-29.663676,4.703570,1.059998,2.580002,123.866091,125.403238,131.633533,115.939324,33.549581,29.549581,129.680225,41.082423,41.758366,22.536021,54.851989,76.257984,-7.451850,-3.023963,-38.076935
1005,AAPL,2021-03-10,119.980003,122.169998,119.449997,121.690002,111760400.0,130.710007,116.209999,1.306833e+08,123.091428,3.645473,-80.514479,4.652857,0.110001,0.659996,123.347946,124.520761,130.382374,115.800482,35.765559,31.765559,128.876926,39.697485,71.428288,2.364152,14.184749,81.092976,-7.501351,-3.065553,-27.814473


In [23]:
x = stock.drop(columns=['Ticker', 'Date'])
y = stock['Close'].shift(-1)

In [24]:
y

0        33.002182
1        33.351227
2        33.405842
3        33.239635
4        33.588676
           ...    
1002    116.360001
1003    121.089996
1004    119.980003
1005    121.959999
1006           NaN
Name: Close, Length: 1007, dtype: float64

In [25]:
x.isna().sum()

Close        0
High         0
Low          0
Open         0
Volume       0
HH           0
LL           0
AV           0
SMA          0
SD           1
WILLR        0
ATR          0
DMH          0
DML          0
EMA          0
WMA         13
BBHIGH       1
BBLOW        1
PERBHIGH     1
PERBLOW      1
TRIMA       13
RSI         14
DX           1
PDI          0
NDI          0
ADX          1
ROC          0
MACD         0
CCI          1
dtype: int64

In [26]:
x.drop(x.head(14).index, inplace=True)
x.drop(x.tail(1).index, inplace=True)
y.drop(y.head(14).index, inplace=True)
y.drop(y.tail(1).index, inplace=True)

In [27]:
x.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)


In [28]:
x

,Close,High,Low,Open,Volume,HH,LL,AV,SMA,SD,WILLR,ATR,DMH,DML,EMA,WMA,BBHIGH,BBLOW,PERBHIGH,PERBLOW,TRIMA,RSI,DX,PDI,NDI,ADX,ROC,MACD,CCI
0,34.111042,34.255882,33.956702,34.125288,78646800.0,34.310500,32.914335,1.030859e+08,33.594775,0.402006,-48.412372,0.386693,-0.054618,-0.116355,33.767732,33.419045,34.398787,32.790764,85.567898,81.567898,33.313598,68.583202,36.109567,-14.124242,-30.089709,103.746181,3.204032,0.102052,56.551631
1,34.120541,34.220266,33.966204,34.122917,79942800.0,34.310500,32.914335,1.044221e+08,33.674658,0.385989,-39.252456,0.391103,-0.035616,0.009502,33.820076,33.508709,34.446636,32.902680,89.242552,85.242552,33.359851,68.695586,172.775066,-9.106582,2.429592,110.134075,3.388741,0.108348,48.196790
2,34.374607,34.403099,33.994697,34.013693,79565600.0,34.403099,32.914335,1.027649e+08,33.747757,0.415780,-6.976455,0.390424,0.182833,0.028493,33.901130,33.561579,34.579317,32.916197,83.167326,79.167326,33.403609,71.619935,73.034207,46.829255,7.297910,110.312172,3.068494,0.124394,48.224358
3,34.196522,34.538440,34.146657,34.244010,110871600.0,34.538440,32.914335,1.051894e+08,33.804234,0.419448,-87.272391,0.405519,0.135341,0.151960,33.943759,33.614698,34.643130,32.965337,82.592144,78.592144,33.446567,66.902242,5.784577,33.374753,37.472995,106.154076,2.366891,0.127019,57.039460
4,34.111042,34.315243,34.061177,34.260629,84596000.0,34.538440,32.914335,9.869340e+07,33.866477,0.393045,-80.373339,0.404841,-0.223197,-0.085480,33.967638,33.698277,34.652568,33.080387,88.164316,84.164316,33.493445,64.699241,44.615374,-55.132055,-21.114409,102.930379,2.621589,0.123572,36.380664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987,120.129997,123.599998,118.620003,121.750000,177275300.0,136.009995,118.389999,1.154656e+08,126.681428,4.699653,-69.678799,4.239285,-2.110001,-3.219994,126.143468,128.795999,136.080733,117.282123,28.955488,24.955488,132.027771,34.709517,20.825407,-49.772564,-75.956062,100.231041,-11.100427,-2.485198,-52.688788
988,121.419998,121.940002,117.570000,120.980003,153590400.0,136.009995,117.570000,1.221485e+08,125.685000,4.164122,-11.899404,4.419999,-1.659996,-1.050003,125.513672,127.504667,134.013243,117.356756,32.182835,28.182835,131.267176,37.633665,22.508974,-37.556478,-23.755729,95.911798,-10.305088,-2.617844,-63.291977
989,116.360001,121.000000,116.209999,120.930000,153918600.0,132.220001,116.209999,1.273872e+08,124.482857,4.259084,-96.868445,4.562142,-0.940002,-1.360001,124.293182,126.343714,133.001024,115.964690,31.227616,27.227616,130.470491,31.646513,18.260766,-20.604410,-29.810572,73.747418,-12.636085,-3.026505,-61.336693
990,121.089996,122.059998,118.790001,119.029999,129159600.0,130.710007,116.209999,1.296187e+08,123.786428,3.923552,-29.663676,4.703570,1.059998,2.580002,123.866091,125.403238,131.633533,115.939324,33.549581,29.549581,129.680225,41.082423,41.758366,22.536021,54.851989,76.257984,-7.451850,-3.023963,-38.076935


In [29]:
x.shape

(992, 29)

In [30]:
y.shape

(992,)

In [31]:
length = x.shape[0]
length

992

In [32]:
trail_size=3

In [33]:
y.head()

0    34.120541
1    34.374607
2    34.196522
3    34.111042
4    34.035061
Name: Close, dtype: float64

In [34]:
y = y.iloc[trail_size-1:-1].reset_index(drop=True)

In [35]:
y.head()

0    34.196522
1    34.111042
2    34.035061
3    33.994701
4    33.629044
Name: Close, dtype: float64

In [36]:
y.shape

(989,)

In [37]:
x_points = [x.iloc[i:i+trail_size].reset_index(drop=True) for i in range(length-trail_size)]

In [38]:
len(x_points)

989

In [39]:
x_points[0]

,Close,High,Low,Open,Volume,HH,LL,AV,SMA,SD,WILLR,ATR,DMH,DML,EMA,WMA,BBHIGH,BBLOW,PERBHIGH,PERBLOW,TRIMA,RSI,DX,PDI,NDI,ADX,ROC,MACD,CCI
0,34.111042,34.255882,33.956702,34.125288,78646800.0,34.310500,32.914335,1.030859e+08,33.594775,0.402006,-48.412372,0.386693,-0.054618,-0.116355,33.767732,33.419045,34.398787,32.790764,85.567898,81.567898,33.313598,68.583202,36.109567,-14.124242,-30.089709,103.746181,3.204032,0.102052,56.551631
1,34.120541,34.220266,33.966204,34.122917,79942800.0,34.310500,32.914335,1.044221e+08,33.674658,0.385989,-39.252456,0.391103,-0.035616,0.009502,33.820076,33.508709,34.446636,32.902680,89.242552,85.242552,33.359851,68.695586,172.775066,-9.106582,2.429592,110.134075,3.388741,0.108348,48.196790
2,34.374607,34.403099,33.994697,34.013693,79565600.0,34.403099,32.914335,1.027649e+08,33.747757,0.415780,-6.976455,0.390424,0.182833,0.028493,33.901130,33.561579,34.579317,32.916197,83.167326,79.167326,33.403609,71.619935,73.034207,46.829255,7.297910,110.312172,3.068494,0.124394,48.224358


In [40]:
y

0       34.196522
1       34.111042
2       34.035061
3       33.994701
4       33.629044
          ...    
984    120.129997
985    121.419998
986    116.360001
987    121.089996
988    119.980003
Name: Close, Length: 989, dtype: float64

In [41]:
x_points[1]

,Close,High,Low,Open,Volume,HH,LL,AV,SMA,SD,WILLR,ATR,DMH,DML,EMA,WMA,BBHIGH,BBLOW,PERBHIGH,PERBLOW,TRIMA,RSI,DX,PDI,NDI,ADX,ROC,MACD,CCI
0,34.120541,34.220266,33.966204,34.122917,79942800.0,34.310500,32.914335,1.044221e+08,33.674658,0.385989,-39.252456,0.391103,-0.035616,0.009502,33.820076,33.508709,34.446636,32.902680,89.242552,85.242552,33.359851,68.695586,172.775066,-9.106582,2.429592,110.134075,3.388741,0.108348,48.196790
1,34.374607,34.403099,33.994697,34.013693,79565600.0,34.403099,32.914335,1.027649e+08,33.747757,0.415780,-6.976455,0.390424,0.182833,0.028493,33.901130,33.561579,34.579317,32.916197,83.167326,79.167326,33.403609,71.619935,73.034207,46.829255,7.297910,110.312172,3.068494,0.124394,48.224358
2,34.196522,34.538440,34.146657,34.244010,110871600.0,34.538440,32.914335,1.051894e+08,33.804234,0.419448,-87.272391,0.405519,0.135341,0.151960,33.943759,33.614698,34.643130,32.965337,82.592144,78.592144,33.446567,66.902242,5.784577,33.374753,37.472995,106.154076,2.366891,0.127019,57.039460


In [42]:
x = np.array(x_points)
y = np.array(y)

In [43]:
x.shape

(989, 3, 29)

In [44]:
y.shape

(989,)

In [45]:
num_features = x.shape[2]

In [66]:
model = Sequential()
model.add(LSTM(units=500, input_shape=(3, 29), return_sequences=True))
model.add(LSTM(units=250, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(1, activation='linear'))

In [67]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 3, 500)            1060000   
_________________________________________________________________
lstm_7 (LSTM)                (None, 3, 250)            751000    
_________________________________________________________________
lstm_8 (LSTM)                (None, 50)                60200     
_________________________________________________________________
dense_5 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_6 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 26        
Total params: 1,875,051
Trainable params: 1,875,051
Non-trainable params: 0
____________________________________________

In [68]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [73]:
model.fit(x, y, epochs=1000)

Epoch 1/1000
31/31 [==============================] - 0s 7ms/step - loss: 294.1752
Epoch 2/1000
31/31 [==============================] - 0s 7ms/step - loss: 290.1137
Epoch 3/1000
31/31 [==============================] - 0s 7ms/step - loss: 268.3350
Epoch 4/1000
31/31 [==============================] - 0s 7ms/step - loss: 251.9921
Epoch 5/1000
31/31 [==============================] - 0s 7ms/step - loss: 227.7950
Epoch 6/1000
31/31 [==============================] - 0s 7ms/step - loss: 223.7508
Epoch 7/1000
31/31 [==============================] - 0s 7ms/step - loss: 211.3858
Epoch 8/1000
31/31 [==============================] - 0s 7ms/step - loss: 209.7041
Epoch 9/1000
31/31 [==============================] - 0s 7ms/step - loss: 205.2350
Epoch 10/1000
31/31 [==============================] - 0s 7ms/step - loss: 230.9250
Epoch 11/1000
31/31 [==============================] - 0s 7ms/step - loss: 208.0886
Epoch 12/1000
31/31 [==============================] - 0s 7ms/step - loss: 180.4078
E

31/31 [==============================] - 0s 7ms/step - loss: 14.7817
Epoch 100/1000
31/31 [==============================] - 0s 7ms/step - loss: 16.1345
Epoch 101/1000
31/31 [==============================] - 0s 7ms/step - loss: 15.1963
Epoch 102/1000
31/31 [==============================] - 0s 7ms/step - loss: 13.4783
Epoch 103/1000
31/31 [==============================] - 0s 7ms/step - loss: 11.2174
Epoch 104/1000
31/31 [==============================] - 0s 7ms/step - loss: 10.8571
Epoch 105/1000
31/31 [==============================] - 0s 7ms/step - loss: 12.7065
Epoch 106/1000
31/31 [==============================] - 0s 7ms/step - loss: 12.8744
Epoch 107/1000
31/31 [==============================] - 0s 7ms/step - loss: 15.5630
Epoch 108/1000
31/31 [==============================] - 0s 7ms/step - loss: 17.5009
Epoch 109/1000
31/31 [==============================] - 0s 7ms/step - loss: 20.9250
Epoch 110/1000
31/31 [==============================] - 0s 7ms/step - loss: 23.8586
Epoch 1

31/31 [==============================] - 0s 7ms/step - loss: 4.5495
Epoch 198/1000
31/31 [==============================] - 0s 7ms/step - loss: 4.6804
Epoch 199/1000
31/31 [==============================] - 0s 7ms/step - loss: 5.8307
Epoch 200/1000
31/31 [==============================] - 0s 7ms/step - loss: 6.0625
Epoch 201/1000
31/31 [==============================] - 0s 7ms/step - loss: 6.6931
Epoch 202/1000
31/31 [==============================] - 0s 7ms/step - loss: 7.3107
Epoch 203/1000
31/31 [==============================] - 0s 7ms/step - loss: 7.8154
Epoch 204/1000
31/31 [==============================] - 0s 7ms/step - loss: 7.7000
Epoch 205/1000
31/31 [==============================] - 0s 7ms/step - loss: 8.6212
Epoch 206/1000
31/31 [==============================] - 0s 7ms/step - loss: 9.0320
Epoch 207/1000
31/31 [==============================] - 0s 7ms/step - loss: 7.5163
Epoch 208/1000
31/31 [==============================] - 0s 7ms/step - loss: 7.5559
Epoch 209/1000
31/3

31/31 [==============================] - 0s 7ms/step - loss: 4.8299
Epoch 296/1000
31/31 [==============================] - 0s 7ms/step - loss: 3.7004
Epoch 297/1000
31/31 [==============================] - 0s 7ms/step - loss: 3.3771
Epoch 298/1000
31/31 [==============================] - 0s 7ms/step - loss: 3.2577
Epoch 299/1000
31/31 [==============================] - 0s 7ms/step - loss: 3.1697
Epoch 300/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.9069
Epoch 301/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.8371
Epoch 302/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.9379
Epoch 303/1000
31/31 [==============================] - 0s 7ms/step - loss: 3.2021
Epoch 304/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.8859
Epoch 305/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.5059
Epoch 306/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.4543
Epoch 307/1000
31/3

31/31 [==============================] - 0s 7ms/step - loss: 2.7176
Epoch 394/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.7811
Epoch 395/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.5474
Epoch 396/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.6243
Epoch 397/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.6863
Epoch 398/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.6662
Epoch 399/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.8223
Epoch 400/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.9216
Epoch 401/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.4097
Epoch 402/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.5138
Epoch 403/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.3860
Epoch 404/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.5685
Epoch 405/1000
31/3

...

31/31 [==============================] - 0s 7ms/step - loss: 2.3551
Epoch 590/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.2987
Epoch 591/1000
31/31 [==============================] - 0s 8ms/step - loss: 2.3882
Epoch 592/1000
31/31 [==============================] - 0s 8ms/step - loss: 2.2413
Epoch 593/1000
31/31 [==============================] - 0s 8ms/step - loss: 2.3378
Epoch 594/1000
31/31 [==============================] - 0s 8ms/step - loss: 2.4362
Epoch 595/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.5909
Epoch 596/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.5296
Epoch 597/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.5069
Epoch 598/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.4714
Epoch 599/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.5931
Epoch 600/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.5148
Epoch 601/1000
31/3

31/31 [==============================] - 0s 7ms/step - loss: 2.1498
Epoch 688/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.1361
Epoch 689/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.1508
Epoch 690/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.1121
Epoch 691/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.1074
Epoch 692/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.1317
Epoch 693/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.3748
Epoch 694/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.1088
Epoch 695/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.3810
Epoch 696/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.4740
Epoch 697/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.2406
Epoch 698/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.4820
Epoch 699/1000
31/3

31/31 [==============================] - 0s 7ms/step - loss: 3.1738
Epoch 786/1000
31/31 [==============================] - 0s 7ms/step - loss: 3.5088
Epoch 787/1000
31/31 [==============================] - 0s 7ms/step - loss: 3.3841
Epoch 788/1000
31/31 [==============================] - 0s 7ms/step - loss: 4.1545
Epoch 789/1000
31/31 [==============================] - 0s 7ms/step - loss: 5.0516
Epoch 790/1000
31/31 [==============================] - 0s 7ms/step - loss: 5.1657
Epoch 791/1000
31/31 [==============================] - 0s 7ms/step - loss: 4.8802
Epoch 792/1000
31/31 [==============================] - 0s 7ms/step - loss: 3.8049
Epoch 793/1000
31/31 [==============================] - 0s 7ms/step - loss: 3.2193
Epoch 794/1000
31/31 [==============================] - 0s 7ms/step - loss: 3.0512
Epoch 795/1000
31/31 [==============================] - 0s 7ms/step - loss: 3.0736
Epoch 796/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.9800
Epoch 797/1000
31/3

31/31 [==============================] - 0s 7ms/step - loss: 2.0527
Epoch 884/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.2204
Epoch 885/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.0709
Epoch 886/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.0031
Epoch 887/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.0711
Epoch 888/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.1353
Epoch 889/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.3405
Epoch 890/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.4696
Epoch 891/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.3884
Epoch 892/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.3197
Epoch 893/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.1259
Epoch 894/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.3281
Epoch 895/1000
31/3

31/31 [==============================] - 0s 7ms/step - loss: 2.7798
Epoch 982/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.6042
Epoch 983/1000
31/31 [==============================] - 0s 8ms/step - loss: 2.7823
Epoch 984/1000
31/31 [==============================] - 0s 7ms/step - loss: 3.0667
Epoch 985/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.8034
Epoch 986/1000
31/31 [==============================] - 0s 8ms/step - loss: 3.3711
Epoch 987/1000
31/31 [==============================] - 0s 7ms/step - loss: 3.2148
Epoch 988/1000
31/31 [==============================] - 0s 7ms/step - loss: 3.0487
Epoch 989/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.8848
Epoch 990/1000
31/31 [==============================] - 0s 8ms/step - loss: 2.5301
Epoch 991/1000
31/31 [==============================] - 0s 7ms/step - loss: 2.9030
Epoch 992/1000
31/31 [==============================] - 0s 7ms/step - loss: 6.0295
Epoch 993/1000
31/3

In [74]:
model.predict(x).flatten().shape

(989,)

In [75]:
res = pd.DataFrame([model.predict(x).flatten(), y]).T

In [76]:
res.describe()

,0,1
count,989.000000,989.000000
mean,62.188646,62.103816
std,29.207997,29.261377
min,31.671629,33.403461
25%,41.541138,41.574829
50%,49.548733,49.855137
75%,74.514122,74.127892
max,143.820709,142.946396


In [77]:
res

,0,1
0,35.834705,34.196522
1,33.947765,34.111042
2,35.543449,34.035061
3,34.263374,33.994701
4,34.909019,33.629044
...,...,...
984,119.176933,120.129997
985,120.547127,121.419998
986,116.754623,116.360001
987,120.928734,121.089996
